In [14]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Correspond to an offline power prediction model to be used to predict the mean 
    power consumption, over the execution interval, of a job submitted on the Erurora HPC. 
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-06-30')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
#infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_359350"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [19]:
# offline
infile_validate_approx = datadir + "CPUs/" + interval_comment + "/validate_pred_off_" + interval_comment + ".csv"

## online
#infile_validate_approx = datadir + "CPUs/" + interval_comment + "/validate_pred_" + interval_comment + ".csv"

# rf
infile_validate_approx = datadir + "CPUs/" + interval_comment + "/validate_pred_rf_" + interval_comment + ".csv"

In [20]:
predicted_power_consumptions = pd.read_csv(infile_validate_approx)

#approximated_power_consumption = approximated_power_consumption[pd.to_datetime(approximated_power_consumption['timestamp']) >= pd.to_datetime('2014-03-31 11:21:00')]
#approximated_power_consumption = approximated_power_consumption[pd.to_datetime(approximated_power_consumption['timestamp']) <= pd.to_datetime('2014-04-30 20:32:00')]
#predicted_power_consumptions = predicted_power_consumptions[pd.to_datetime(predicted_power_consumptions['timestamp']) >= pd.to_datetime('2014-10-01 11:21:00')]

# October has only 10 days of data
predicted_power_consumptions = predicted_power_consumptions[pd.to_datetime(predicted_power_consumptions['timestamp']) <= pd.to_datetime('2014-10-07 18:30:00')]

#approx = approximated_power_consumption.select(lambda col: col.endswith("approx"), axis=1)
real = predicted_power_consumptions.select(lambda col: col.endswith("real"), axis=1)
predicted = predicted_power_consumptions.select(lambda col: col.endswith("pred"), axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  if sys.path[0] == '':


In [21]:
real['Total'] = real.iloc[:, -64:-1].sum(axis=1)
predicted['Total'] = predicted.iloc[:, -64:-1].sum(axis=1)

In [22]:
mean_real_pow = real['Total'].mean()
print("the mean is {}".format(mean_real_pow))

numerator = (real['Total'] - predicted['Total'])**2
denumerator = (real['Total'] - mean_real_pow)**2
N = numerator.shape[0]

r_2 = 1 - (numerator.sum() / denumerator.sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(numerator.sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 6352.353895830268
r_2 is 0.9483753018370696
RMSE is 788.5119556912498
NRMSE is 0.12412909743722479
